In [1]:
!pip install gradio plotly requests

In [2]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import os
import plotly.express as px
import gradio as gr

USGS_DAY_URL = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson"
USGS_MONTH_URL = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson"


In [3]:
def fetch_earthquake_data(url: str = USGS_DAY_URL) -> pd.DataFrame:
    """
    Fetch earthquake data from USGS GeoJSON feed and return a clean DataFrame.
    """
    resp = requests.get(url, timeout=10)
    resp.raise_for_status()
    data = resp.json()

    records = []
    for feature in data.get("features", []):
        props = feature.get("properties", {})
        geom = feature.get("geometry", {})
        coords = geom.get("coordinates", [None, None, None])
        lon, lat, depth_km = (coords + [None, None, None])[:3]

        quake_time_ms = props.get("time")
        dt = datetime.fromtimestamp(quake_time_ms / 1000.0, tz=timezone.utc) if quake_time_ms else None

        records.append({
            "time_utc": dt,
            "magnitude": props.get("mag"),
            "place": props.get("place"),
            "longitude": lon,
            "latitude": lat,
            "depth_km": depth_km,
            "tsunami_flag": props.get("tsunami"),
            "alert_level_raw": props.get("alert"),
            "felt_reports": props.get("felt"),
            "url": props.get("url")
        })

    df = pd.DataFrame(records)
    # Drop rows with missing mag / coords
    df = df.dropna(subset=["magnitude", "latitude", "longitude"]).reset_index(drop=True)
    return df


In [4]:
def classify_severity(mag: float) -> str:
    if mag < 2.0:
        return "Micro"
    elif mag < 4.0:
        return "Minor"
    elif mag < 5.0:
        return "Light"
    elif mag < 6.0:
        return "Moderate"
    elif mag < 7.0:
        return "Strong"
    elif mag < 8.0:
        return "Major"
    else:
        return "Great"


def classify_depth(depth_km: float) -> str:
    if depth_km is None:
        return "Unknown"
    if depth_km < 70:
        return "Shallow"
    elif depth_km < 300:
        return "Intermediate"
    else:
        return "Deep"


def extract_region(place: str) -> str:
    if not isinstance(place, str):
        return "Unknown"
    parts = place.split(",")
    if len(parts) > 1:
        region = parts[-1].strip()
    else:
        region = place.strip()
    return region if region else "Unknown"


def compute_risk_score(mag: float, depth_km: float) -> float:
    if depth_km is None or depth_km <= 0:
        depth_km = 10.0
    if depth_km < 70:
        depth_factor = 1.3
    elif depth_km < 300:
        depth_factor = 1.1
    else:
        depth_factor = 0.9
    return round(mag * depth_factor, 2)


def enrich_earthquake_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["severity"] = df["magnitude"].apply(classify_severity)
    df["depth_category"] = df["depth_km"].apply(classify_depth)
    df["region"] = df["place"].apply(extract_region)
    df["risk_score"] = df.apply(lambda r: compute_risk_score(r["magnitude"], r["depth_km"]), axis=1)

    def risk_label(score):
        if score >= 7.0:
            return "Critical"
        elif score >= 5.5:
            return "High"
        elif score >= 4.0:
            return "Moderate"
        else:
            return "Low"
    df["risk_level"] = df["risk_score"].apply(risk_label)
    df = df.sort_values("time_utc", ascending=False).reset_index(drop=True)
    return df

In [5]:
def export_view_to_csv(df: pd.DataFrame, prefix: str = "earthquakes_view") -> str:
    """
    Save DataFrame as CSV in /tmp and return the path.
    """
    os.makedirs("/tmp", exist_ok=True)
    path = f"/tmp/{prefix}.csv"
    df.to_csv(path, index=False)
    return path

In [6]:
def build_live_dashboard(
    min_mag: float = 3.0,
    region: str = "All",
    sort_by: str = "Newest"
):
    # 1) Fetch + enrich
    base_df = fetch_earthquake_data(USGS_DAY_URL)
    df = enrich_earthquake_data(base_df)

    # 2) Filter
    df = df[df["magnitude"] >= min_mag].copy()
    if region != "All":
        df = df[df["region"] == region].copy()

    # If nothing left after filtering
    if df.empty:
        summary_md = f"### No earthquakes found for the selected filters in the last 24 hours."
        alert_md = "✅ No major earthquakes (M ≥ 6.0) in current view."
        fig = px.scatter(title="No data")
        empty_df = pd.DataFrame()
        csv_path = export_view_to_csv(empty_df, prefix="earthquakes_empty_view")
        return summary_md, alert_md, fig, empty_df, csv_path

    # 3) Summary text
    total_quakes = len(df)
    max_mag = df["magnitude"].max()
    # IMPORTANT: use loc (label), not iloc (position)
    strongest = df.loc[df["magnitude"].idxmax()]
    strongest_place = strongest["place"]
    strongest_time = strongest["time_utc"].strftime("%Y-%m-%d %H:%M:%S UTC") if strongest["time_utc"] else "Unknown"

    risk_counts = df["risk_level"].value_counts().to_dict()

    summary_lines = [
        f"### Earthquake Summary (last 24 hours, mag ≥ {min_mag})",
        f"- Total earthquakes: **{total_quakes}**",
        f"- Strongest event: **M {max_mag:.1f}** near **{strongest_place}**",
        f"- Time of strongest event: **{strongest_time}**",
        "",
        "#### Risk Level Distribution:"
    ]
    for level in ["Critical", "High", "Moderate", "Low"]:
        count = risk_counts.get(level, 0)
        summary_lines.append(f"- {level}: **{count}**")
    summary_md = "\n".join(summary_lines)

    # 4) Alert rule (M ≥ 6)
    major_quakes = df[df["magnitude"] >= 6.0]
    if not major_quakes.empty:
        strongest_major = major_quakes.loc[major_quakes["magnitude"].idxmax()]
        alert_md = (
            f"🚨 **Major earthquake detected!**\n\n"
            f"- Max magnitude: **M {strongest_major['magnitude']:.1f}**\n"
            f"- Location: **{strongest_major['place']}**\n"
            f"- Time: **{strongest_major['time_utc']} (UTC)**\n\n"
            "Review seismic activity and follow official emergency channels."
        )
    else:
        alert_md = "✅ No major earthquakes (M ≥ 6.0) in current view."

    # 5) Plot
    df_plot = df.copy()
    df_plot["time_str"] = df_plot["time_utc"].astype(str)
    fig = px.scatter(
        df_plot,
        x="time_str",
        y="magnitude",
        size="risk_score",
        color="risk_level",
        hover_data=["place", "depth_km", "region"],
        title="Earthquake Magnitude Over Time (Bubble size = risk score)",
        labels={"time_str": "Time (UTC)", "magnitude": "Magnitude"}
    )
    fig.update_layout(xaxis_tickangle=-45)

    # 6) Sort table
    if sort_by == "Newest":
        df_table = df.sort_values("time_utc", ascending=False)
    elif sort_by == "Magnitude (High→Low)":
        df_table = df.sort_values("magnitude", ascending=False)
    elif sort_by == "Risk (High→Low)":
        df_table = df.sort_values("risk_score", ascending=False)
    else:
        df_table = df

    df_table = df_table[[
        "time_utc", "magnitude", "severity", "region",
        "depth_km", "depth_category", "risk_score", "risk_level",
        "place", "url"
    ]].head(200)

    csv_path = export_view_to_csv(df_table, prefix="earthquakes_live_view")

    return summary_md, alert_md, fig, df_table, csv_path

In [7]:
def top10_major_earthquakes_month():
    """
    Top 10 strongest earthquakes in the last 30 days (by place/city/region).
    """
    df_month = fetch_earthquake_data(USGS_MONTH_URL)
    df_month = enrich_earthquake_data(df_month)

    if df_month.empty:
        return "No data for this month.", px.bar(title="No data"), pd.DataFrame()

    df_top = df_month.sort_values("magnitude", ascending=False).head(10)
    df_top = df_top[[
        "time_utc", "magnitude", "severity", "region",
        "depth_km", "depth_category", "risk_score", "risk_level",
        "place", "url"
    ]]

    fig = px.bar(
        df_top.sort_values("magnitude"),
        x="magnitude",
        y="place",
        orientation="h",
        color="risk_level",
        title="Top 10 Major Earthquakes (Last 30 Days)",
        labels={"magnitude": "Magnitude", "place": "Location / City / Region"}
    )

    return "### Top 10 Major Earthquakes (Last 30 Days)", fig, df_top

In [8]:
def get_region_choices():
    df = enrich_earthquake_data(fetch_earthquake_data(USGS_DAY_URL))
    regions = sorted(df["region"].dropna().unique().tolist())
    return ["All"] + regions

In [11]:
def live_callback(min_mag, region, sort_by):
    return build_live_dashboard(min_mag=min_mag, region=region, sort_by=sort_by)

def top10_callback():
    return top10_major_earthquakes_month()


with gr.Blocks(title="Real-Time Earthquake Alert & Analytics System") as demo:
    gr.Markdown(
        """
        # QuakeSense – Real-Time Seismic Analytics System
        Live and recent analytics of global earthquakes.
        *Data source: USGS Earthquake Hazards Program.*
        """
    )

    with gr.Tab("Live Dashboard (24h)"):
        with gr.Row():
            min_mag_slider = gr.Slider(0.0, 8.0, value=3.0, step=0.1, label="Minimum Magnitude")
            region_dropdown = gr.Dropdown(choices=["All"], value="All", label="Region (auto-filled)")
            sort_dropdown = gr.Dropdown(
                choices=["Newest", "Magnitude (High→Low)", "Risk (High→Low)"],
                value="Newest",
                label="Sort Table By"
            )
        refresh_btn = gr.Button("🔄 Refresh Data & Update Dashboard")

        alert_out = gr.Markdown(label="Alert")
        summary_out = gr.Markdown(label="Summary")
        plot_out = gr.Plot(label="Magnitude vs Time")
        table_out = gr.Dataframe(label="Earthquake Events", interactive=False, wrap=True)
        csv_out = gr.File(label="Download Filtered View (CSV)")

        # init region list on load
        def init_regions():
            return gr.update(choices=get_region_choices(), value="All")

        demo.load(init_regions, inputs=None, outputs=region_dropdown)

        refresh_btn.click(
            fn=live_callback,
            inputs=[min_mag_slider, region_dropdown, sort_dropdown],
            outputs=[summary_out, alert_out, plot_out, table_out, csv_out]
        )

    with gr.Tab("Top 10 Major Earthquakes (30 days)"):
        top10_btn = gr.Button("🔄 Load Top 10")
        top10_summary = gr.Markdown()
        top10_plot = gr.Plot()
        top10_table = gr.Dataframe(interactive=False, wrap=True)

        top10_btn.click(
            fn=top10_callback,
            inputs=None,
            outputs=[top10_summary, top10_plot, top10_table]
        )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a910be1189749ac26e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
